### 
- Fatima 
- Israa zaher yahya 201910063
- Battol

In [1]:
import time

In [2]:
start_time=time.time()

In [3]:
import pandas as pd


In [4]:
#find the path to cases datafram
file_path1 ="C:/Users/yafa/Desktop/df_cases_200906/df_cases_200906.gzip" 

In [5]:
#read the cases data frame from the path 
df_cases = pd.read_pickle(file_path1)

In [6]:
#print the cases dataframe 
df_cases

,CaseId,FileName,Language,StartDate,DocumentType,IsExecuted,OcrText,QualityScore
0,003061226227,003061226227_12045631_Order form_978-1-67767-3...,EN,20191104,Order form,False,Pressure style response character.,0.649292
1,003061226383,003061226383_63912371_Contract Info Pack_978-0...,EN,20190805,Contract Info Pack,False,Soon especially boy thousand traditional.,0.873038
2,003061226383,003061226383_41775424_other documents_978-1-38...,EN,20190805,other documents,True,Tax south say strategy hard between late.,0.991280
3,003061227777,003061227777_96463321_other documents_978-1-96...,EN,20191231,other documents,False,Yet arm six design.,0.607833
4,003061227781,003061227781_62201704_other documents_978-1-04...,EN,20191231,other documents,False,Plan camera actually run push rest.,0.909359
...,...,...,...,...,...,...,...,...
2328,003061208849,003061208849_91360175_Order form_978-0-00-9102...,EN,20200331,Order form,True,Woman month tough similar natural choice east.,0.693007
2329,003061207708,003061207708_56075304_Order form_978-1-952184-...,EN,20191201,Order form,True,Against use break impact.,0.983739
2330,003061207200,003061207200_62475099_Order form_978-0-918578-...,EN,20190701,Order form,True,Week determine entire newspaper church maintai...,0.745751
2331,003061209028,003061209028_76788956_Order form_978-1-67079-8...,EN,20191201,Order form,True,Ten kitchen word start give service oil.,0.713114


In [7]:
#find the path to the lable data frame 
file_path2 ="C:/Users/yafa/Desktop/df_cases_200906/two.gzip" 

In [8]:
#read the data frame from the path 
df_lable= pd.read_pickle(file_path2)

In [9]:
#print the lable data frame
df_lable

,CaseId,label_1,label_2
0,003061226227,False,False
1,003061226383,True,False
2,003061227777,False,False
3,003061227781,False,False
4,003061227680,True,True
...,...,...,...
1093,003061208632,True,False
1094,003061208849,False,False
1095,003061207708,False,False
1096,003061207200,True,False


In [10]:
#The main objective is to merge `label_1` and `label_2` => into `df_cases_200906.gzip`,
df_cases = pd.merge(df_cases, df_lable, on='CaseId')

In [11]:
#df_cases

In [12]:
# Step 1: create a new  dataframe filter the cases dataframe to keep only valid contracts

df_valid_contracts = df_cases[(df_cases['IsExecuted'] == True) & (df_cases['QualityScore'] >= 0.81)]

In [13]:
#print the valid contract
#df_valid_contracts

In [14]:
# Step 2: group by CaseId
groups = df_valid_contracts.groupby('CaseId')

In [15]:
# Step 3: concatenate OcrText fields for each group
concatenated_text = groups['OcrText'].apply(lambda x: '\n'.join(x)).reset_index()

In [16]:
# Step 4: create a new dataframe with CaseId and ConcatenatedOcrText columns
df_concatenated = pd.DataFrame({'CaseId': concatenated_text['CaseId'], 'ConcatenatedOcrText': concatenated_text['OcrText']})

In [17]:
df_concatenated

,CaseId,ConcatenatedOcrText
0,003061189006,As difficult behavior her myself help.
1,003061189067,Present can phone form.
2,003061189156,Section science difference success wish it wide.
3,003061189229,Accept sell leader herself if.
4,003061189349,Another later everybody large real.
...,...,...
565,003061230596,Miss style unit.\nHigh begin purpose interesti...
566,003061230613,Really certainly might responsibility responsi...
567,003061230657,Cultural light carry past technology finish la...
568,003061230728,None attorney spend tend miss appear.


In [18]:
df_concatenated.rename(columns={'ConcatenatedOcrText': 'OcrText'}, inplace=True)


In [19]:
df_concatenated

,CaseId,OcrText
0,003061189006,As difficult behavior her myself help.
1,003061189067,Present can phone form.
2,003061189156,Section science difference success wish it wide.
3,003061189229,Accept sell leader herself if.
4,003061189349,Another later everybody large real.
...,...,...
565,003061230596,Miss style unit.\nHigh begin purpose interesti...
566,003061230613,Really certainly might responsibility responsi...
567,003061230657,Cultural light carry past technology finish la...
568,003061230728,None attorney spend tend miss appear.


In [20]:
#function 
def get_filenames(group):
    filenames = group['FileName'].tolist()
   
    valid_filenames = []
    invalid_filenames = []
   
    for filename in filenames:
        #check the condition of valid contract 
        if group.loc[group['FileName'] == filename, 'IsExecuted'].iloc[0] and group.loc[group['FileName'] == filename, 'QualityScore'].iloc[0] >= 0.81:
            valid_filenames.append(filename)
        else:
            invalid_filenames.append(filename)
            
    return valid_filenames, invalid_filenames



In [21]:
df_new=df =pd.DataFrame()

In [22]:
df_new[['ValidFileNames', 'InvalidFileNames']] = df_cases.groupby('CaseId').apply(get_filenames).apply(pd.Series)

In [23]:
df_new

,ValidFileNames,InvalidFileNames
CaseId,,
003061189006,[003061189006_69176036_Order form_978-1-62414-...,[]
003061189067,[003061189067_26173467_Order form_978-1-904782...,[]
003061189156,[003061189156_47966765_Contract Documents_978-...,[]
003061189229,[003061189229_21094545_other documents_978-1-8...,[003061189229_69115288_Master contract_978-0-1...
003061189242,[],[003061189242_91658807_other documents_978-0-7...
...,...,...
003061230659,[],[003061230659_72651667_Order form_978-0-661-06...
003061230710,[],[003061230710_80047544_other documents_978-1-0...
003061230728,[003061230728_74076581_Amendments_978-0-14-763...,[003061230728_79408066_Master contract_978-0-1...


In [24]:
df =pd.DataFrame(df_cases.loc[:, ['CaseId', 'label_1', 'label_2', ]])

In [25]:
df = df.drop_duplicates()

In [26]:
df

,CaseId,label_1,label_2
0,003061226227,False,False
3,003061226383,True,False
8,003061227777,False,False
10,003061227781,False,False
12,003061227680,True,True
...,...,...,...
2063,003061208632,True,False
2065,003061208849,False,False
2066,003061207708,False,False
2067,003061207200,True,False


In [27]:
df_final = pd.merge(df_new, df_concatenated, on='CaseId')

In [28]:
df_final = pd.merge(df_final, df, on='CaseId')

In [29]:
df_final

,CaseId,ValidFileNames,InvalidFileNames,OcrText,label_1,label_2
0,003061189006,[003061189006_69176036_Order form_978-1-62414-...,[],As difficult behavior her myself help.,True,False
1,003061189067,[003061189067_26173467_Order form_978-1-904782...,[],Present can phone form.,True,False
2,003061189156,[003061189156_47966765_Contract Documents_978-...,[],Section science difference success wish it wide.,False,False
3,003061189229,[003061189229_21094545_other documents_978-1-8...,[003061189229_69115288_Master contract_978-0-1...,Accept sell leader herself if.,False,False
4,003061189349,[003061189349_72196625_Other documents_978-0-6...,[],Another later everybody large real.,False,False
...,...,...,...,...,...,...
565,003061230596,[003061230596_91328991_other documents_978-0-8...,[003061230596_30702523_other documents_978-0-6...,Miss style unit.\nHigh begin purpose interesti...,False,False
566,003061230613,[003061230613_55291460_other documents_978-1-1...,[003061230613_47613577_Terms & Conditions (all...,Really certainly might responsibility responsi...,False,False
567,003061230657,[003061230657_50860326_Order form_978-1-80159-...,[],Cultural light carry past technology finish la...,True,False
568,003061230728,[003061230728_74076581_Amendments_978-0-14-763...,[003061230728_79408066_Master contract_978-0-1...,None attorney spend tend miss appear.,True,False


In [30]:
df_final.to_csv('C://Users//yafa//Downloads//WPy64-3850//submit//df_final.gzip',index=False, compression='gzip')

C:\Users\yafa\Downloads\WPy64-3850\python-3.8.5.amd64\lib\site-packages\pandas\core\generic.py:3167: RuntimeWarning: compression has no effect when passing file-like object as input.
  formatter.save()


In [31]:
end_time=time.time()

In [32]:
print(f"Excution Time: {end_time-start_time} seconds")

Excution Time: 7.122096538543701 seconds
